In [ ]:
import os
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

In [ ]:
# Configuration(Importing Model Paths)
image_path = r"C:\Users\anish\OneDrive\Desktop\light_vishal.webp" #Update path to your image
predictor_path = r"C:\Users\anish\OneDrive\Desktop\floos\shape_predictor_68_face_landmarks.dat" #Update path to the face prediction landmarks

In [ ]:
# Validate file paths 
if not os.path.isfile(image_path):
    print("❌ Image file not found.")
    exit()

if not os.path.isfile(predictor_path):
    print("❌ Shape predictor file not found.")
    exit()

In [ ]:
# Load models(Dlib)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

In [ ]:
# Read image and detect face 
image = cv2.imread(image_path)
if image is None:
    print("❌ Could not read the image file.")
    exit()

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = detector(gray)

if len(faces) == 0:
    print("❌ No face detected.")
    exit()

face = faces[0]
x, y, w, h = face.left(), face.top(), face.width(), face.height()
face_crop = image[y:y+h, x:x+w]
face_copy = face_crop.copy()
landmarks = predictor(gray, face)

In [ ]:
#  Mask eyes, lips, eyebrows 
def mask_features(indices_list):
    for indices in indices_list:
        pts = np.array([(landmarks.part(i).x - x, landmarks.part(i).y - y) for i in indices], dtype=np.int32)
        cv2.fillPoly(face_copy, [pts], (0, 0, 0))

regions_to_mask = [
    range(36, 42),  # Left eye
    range(42, 48),  # Right eye
    range(48, 61),  # Lips
    range(17, 22),  # Left eyebrow
    range(22, 27)   # Right eyebrow
]

mask_features(regions_to_mask)

In [ ]:
# Extract skin region 
rgb_image = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
skin_mask = np.all(rgb_image != [0, 0, 0], axis=-1)

lab_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2LAB)
pixels = lab_image.reshape(-1, 3)[skin_mask.flatten()]

In [ ]:
# KMeans clustering on LAB pixels 
kmeans = KMeans(n_clusters=8, random_state=42)
labels = kmeans.fit_predict(pixels)
centroids = kmeans.cluster_centers_

In [ ]:
# Compare with reference skin tone LAB values 
reference = {
    "Light": [85, 0, 15],
    "Medium": [65, 10, 20],
    "Dark": [35, 15, 30]
}
ref_values = np.array(list(reference.values()))
label_to_tone = {}

for idx, center in enumerate(centroids):
    distances = cdist([center], ref_values, 'euclidean')
    closest = np.argmin(distances)
    label_to_tone[idx] = list(reference.keys())[closest]

dominant_label = np.bincount(labels).argmax()
skin_tone = label_to_tone[dominant_label]

In [ ]:
# Ask user for undertone 
undertone = input("Enter your undertone (Warm / Cool): ").strip().lower()
while undertone not in ['warm', 'cool']:
    undertone = input("Please type 'Warm' or 'Cool': ").strip().lower()

undertone_label = "Warm" if undertone == "warm" else "Cool"
final_classification = f"{skin_tone}-{undertone_label}" 

In [ ]:
# === Output ===
print("\n Dominant Skin Tone:", skin_tone)
print("  Undertone:", undertone_label)
print(" Final Classification:", final_classification)

In [ ]:
# === Visualization ===
visual = np.zeros((skin_mask.shape[0], skin_mask.shape[1], 3), dtype=np.uint8)
colors = (plt.cm.jet(np.linspace(0, 1, 8))[:, :3] * 255).astype(np.uint8)
label_idx = 0

for i in range(skin_mask.shape[0]):
    for j in range(skin_mask.shape[1]):
        if skin_mask[i, j]:
            visual[i, j] = colors[labels[label_idx]]
            label_idx += 1

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(rgb_image)
plt.title("Masked Face")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(visual)
plt.title(f"{skin_tone} - {undertone_label}")
plt.axis('off')

plt.tight_layout()
plt.show()